In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import azure_recognizer as ar
import azure.cognitiveservices.speech as sdk
import numpy as np
import soundfile as sf
from pathlib import Path
import random
from IPython.display import Audio
import pandas as pd
import time
import matplotlib.pyplot as plt

In [3]:
file_path = "/work/pi_amillner_access-ci_org/kid-whisper/myst_data/myst_child_conv_speech/data/development/004031/myst_004031_2013-12-02_10-20-36_EE_1.3/myst_004031_2013-12-02_10-20-36_EE_1.3_002.flac"
trn_path =  "/work/pi_amillner_access-ci_org/kid-whisper/myst_data/myst_child_conv_speech/data/development/004031/myst_004031_2013-12-02_10-20-36_EE_1.3/myst_004031_2013-12-02_10-20-36_EE_1.3_002.trn"

In [4]:
l60 = Path("/work/pi_amillner_access-ci_org/bu-wise-scope-2026/data/lesson_60")
l61 = Path("/work/pi_amillner_access-ci_org/bu-wise-scope-2026/data/lesson_61")
j_pairs = []
for path in list(l60.rglob("*.wav")) + list(l61.rglob("*.wav")):
    transcription = path.stem.split(" ")[0]
    if "incorrect" in path.stem:
        continue
    if len(transcription) == 1:
        # Skip single phoneme
        continue
    if "-" in transcription:
        # Skip multi-word transcriptions
        continue
    j_pairs.append((path, transcription))

In [22]:
# train_path = Path("/work/pi_amillner_access-ci_org/kid-whisper/myst_data/myst_child_conv_speech/data/development")

# all_pairs = [
#     (flac_path, flac_path.with_suffix(".trn"))
#     for flac_path in train_path.rglob("*.flac")
#     if flac_path.with_suffix(".trn").exists()
# ]

# sample_size = 100
# random.seed(42)  # for reproducibility
# sampled_pairs = random.sample(all_pairs, sample_size)

# for audio, transcript in sampled_pairs[:3]:
#     print("Audio:", audio)
#     print("Transcript:", transcript)

In [ ]:
# MAX_ATTEMPTS = 10
# all_res = []
# for wav, text in j_pairs:
#     for _ in range(MAX_ATTEMPTS):
#         try:
#             res = recognize_from_wav(str(wav), text)
#             pa_result = sdk.PronunciationAssessmentResult(res)
#             acc = pa_result.accuracy_score
#             print(f"{wav.stem}: {acc}")
#             all_res.append({
#                 "lesson": wav.parent.stem,
#                 "filename": wav.stem,
#                 "accuracy": acc,
#                 "completeness": pa_result.completeness_score,
#                 "fluency": pa_result.fluency_score,
#                 "pronunciation": pa_result.pronunciation_score,
#                 "prosody": pa_result.prosody_score,
#             })
#             break
#         except Exception as e:
#             time.sleep(0.1)

In [28]:
res_df = pd.DataFrame(all_res)
res_df.to_csv("l_60_61_results_11_5.csv", index=False)
# res_df

In [37]:
# # which failed?
# for f, _ in j_pairs:
#     if f.stem not in res_df["filename"].values:
#         print(f)

In [5]:
# wav, text = j_pairs[11]

file = "time 2"
wav, text = [p for p in j_pairs if file in p[0].stem][0]

# wav = "/work/pi_amillner_access-ci_org/bu-wise-scope-2026/julianne_recordings_data/lesson_61/i-like-to-go-to-soccer-games 1.wav"
# text = "i like to go to soccer games"

# wav = "/work/pi_amillner_access-ci_org/bu-wise-scope-2026/data/lesson_61/o 1.wav"

res = ar.recognize_from_wav(str(wav), text)
pa_result = ar.display_result(res)
Audio(wav)


=== PRONUNCIATION ASSESSMENT ===
Accuracy Score:       89.0
Completeness Score:   100.0
Fluency Score:        100.0
Pronunciation Score:  75.6
Prosody Score:        44.6

=== WORD-LEVEL DETAILS ===
time: accuracy=89.0, error=None
  t: accuracy=100.0
  ay: accuracy=76.0
  m: accuracy=80.0
